In [9]:
import pandas as pd
import numpy as np
import os
import sys 

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

original_file = "blindern_data_cleaning.csv"
data_path = os.path.join(os.getcwd(),"..", "data")
original_path = os.path.join(data_path, original_file)  
df = pd.read_csv(original_path, delimiter=";")

# calculate the average, median and standard deviation for each column
print(f"The average for each column is:\n{df.mean(numeric_only=True).round(2)}")
print(f"The median for each column is:\n{df.median(numeric_only=True).round(2)}")
print(f"The standard deviation for each column is:\n{df.std(numeric_only=True).round(2)}")

# calculate the seasonal average, median and standard deviation for each column
def season(dato):
    month=dato.month
    if month in [12,1,2]:
        return "Winter"
    elif month in [3,4,5]:
        return "Spring"
    elif month in [6,7,8]:
        return "Summer"
    elif month in [9,10,11]:
        return "Autumn"

df['season'] = df['Tid'].apply(season)



print(f"The average for each season is:\n{df.groupby('season').mean(numeric_only=True).round(2)}")
print(f"The median for each season is:\n{df.groupby('season').median(numeric_only=True).round(2)}")
print(f"The standard deviation for each season is:\n{df.groupby('season').std(numeric_only=True).round(2)}")





The average for each column is:
Makstemp         11.60
Mintemp           4.15
Middeltemp        7.62
Snø               3.97
Nedbør            2.42
Middelvind        2.70
Høye vindkast     5.56
dtype: float64
The median for each column is:
Makstemp         11.4
Mintemp           3.6
Middeltemp        7.1
Snø               0.0
Nedbør            0.1
Middelvind        2.5
Høye vindkast     5.2
dtype: float64
The standard deviation for each column is:
Makstemp         9.05
Mintemp          7.24
Middeltemp       7.86
Snø              9.93
Nedbør           5.26
Middelvind       1.13
Høye vindkast    2.23
dtype: float64
The average for each season is:
        Makstemp  Mintemp  Middeltemp    Snø  Nedbør  Middelvind  \
season                                                             
Autumn     10.91     4.86        7.62   0.12    2.94        2.69   
Spring     11.81     2.64        6.93   4.47    1.68        2.92   
Summer     22.16    12.49       16.92   0.00    3.06        2.70   
Winter  

In [1]:
from statistics import mode

#List of days with snow back to back, only those over X ammount of days.
def snowdays(grense=5):
	count=0
	snowdays=[]
	snowdays_dates=[]
	snow_period=()

	for i in range(len(df)-1):
		if df["Snø"][i] > 0:
			count += 1
		else:
			if count >= grense:                  #X ammont of days
				snowdays.append(count)
				end_date=pd.to_datetime(df["Tid"][i-1]).date()
				start_date=end_date-pd.Timedelta(days=count-1)
				snow_period=f"{start_date} --- {end_date}"
				snowdays_dates.append(snow_period)
			count=0

	print(f"Antall dager med snø etter en annen: {snowdays}")
	print(f"Typetall for antall dager med snø sammenhengende: {mode(snowdays)}\nMinste antall dager er {grense}")
	print(f"Dager med snø, datoer: {snowdays_dates}")

	
#List of days without rain back to back, only those over X ammount of days.
def drydays(grense=12):
	count=0
	no_rain_days=[]
	no_rain_dates=[]
	dry_period=()

	for i in range(len(df)-1):
		if df["Nedbør"][i] <= 0:
			count += 1
		else:
			if count >=grense:                             #X ammont of days
				no_rain_days.append(count)
				end_date=pd.to_datetime(df["Tid"][i-1]).date()
				start_date=end_date-pd.Timedelta(days=count-1)
				dry_period=f"{start_date} --- {end_date}"
				no_rain_dates.append(dry_period)
			count=0
			
	print(f"Antall dager med snø etter en annen: {no_rain_days}")
	print(f"Typetall for antall dager uten nedbør sammenhengende: {mode(no_rain_days)}\nMinste antall dager er {grense}")
	print (f"No rain dates: {no_rain_dates}")

snowdays(5)
drydays()


NameError: name 'df' is not defined